### Heroes Of Pymoli Data Analysis
* Of the 1163 active players, the vast majority are male (84%). There also exists, a smaller, but notable proportion of female players (14%).

* Our peak age demographic falls between 20-24 (44.8%) with secondary groups falling between 15-19 (18.60%) and 25-29 (13.4%).  
-----

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


## Player Count

In [2]:
total_num = purchase_data["SN"].count()
df_total_num = pd.DataFrame({"Total Players": [total_num]})
df_total_num

,Total Players
0,780


## Purchasing Analysis (Total)

### Generate a general summary table displaying: 
1. The Number of Unique Items;
2. Average Price;
3. Number of Purchases; 
4. Total Revenue

In [3]:
num_items = len(purchase_data["Item ID"].unique())
avg_price = purchase_data["Price"].mean()
num_purchases = purchase_data["Purchase ID"].count()
total_revenue = purchase_data["Price"].sum()
df_pa = pd.DataFrame({"Number of Unique Items":[num_items], "Average Price":[avg_price], 
                                      "Number of Purchases":[num_purchases], "Total Revenue":[total_revenue]})
df_pa["Average Price"]=df_pa["Average Price"].astype(float).map("${:.2f}".format)
df_pa["Total Revenue"]=df_pa["Total Revenue"].astype(float).map("${:.2f}".format)
df_pa

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,$2379.77


## Gender Demographics

### Generate a summary table describing the pattern of players with respect to gender demographics
1. Percentage and Count of Male Players
2. Percentage and Count of Female Players
3. Percentage and Count of Other / Non-Disclosed

In [4]:
num_male = purchase_data.loc[purchase_data["Gender"] == "Male"]["Purchase ID"].count()
per_male = num_male / total_num * 100
num_female = purchase_data.loc[purchase_data["Gender"] == "Female"]["Purchase ID"].count()
per_female = num_female / total_num * 100
num_other = purchase_data.loc[purchase_data["Gender"] == "Other / Non-Disclosed"]["Purchase ID"].count()
per_other = num_other / total_num * 100
row_name = ["Male", "Female", "Other / Non-Disclosed"]
df_gender = pd.DataFrame({"Total Count":[num_male, num_female, num_other], 
                         "Percentage of Players":[per_male, per_female, per_other]}, index=row_name)
df_gender["Percentage of Players"] = df_gender["Percentage of Players"].astype(float).map("{:.2f}".format)
df_gender

,Total Count,Percentage of Players
Male,652,83.59
Female,113,14.49
Other / Non-Disclosed,15,1.92


### Purchasing Analysis (Gender)

In [45]:
purchase_data["Price"] = pd.to_numeric(purchase_data["Price"])
df_pa_count = purchase_data.groupby(["Gender"])["Purchase ID"].count().reset_index()
df_pa_price = purchase_data.groupby(["Gender"])["Price"].mean().reset_index()
df_pa_total = purchase_data.groupby(["Gender"])["Price"].sum().reset_index()
df_per_male = (purchase_data.loc[purchase_data["Gender"] == "Male"]["Price"].sum()) / (len(purchase_data.loc[purchase_data["Gender"] == "Male"]["SN"].unique()))
df_per_female = (purchase_data.loc[purchase_data["Gender"] == "Female"]["Price"].sum()) / (len(purchase_data.loc[purchase_data["Gender"] == "Female"]["SN"].unique()))
df_per_other = (purchase_data.loc[purchase_data["Gender"] == "Other / Non-Disclosed"]["Price"].sum()) / (len(purchase_data.loc[purchase_data["Gender"] == "Other / Non-Disclosed"]["SN"].unique()))
df_per_price = pd.DataFrame({"Avg Total Purchase per Person": [df_per_female, df_per_male, df_per_other]})
df_combined = pd.merge(df_pa_count, df_pa_price, on="Gender", how="outer")
df_combined = pd.merge(df_combined, df_pa_total, on="Gender", how="outer")
df_combined["Avg Total Purchase per Person"] = df_per_price
df_combined = df_combined.rename(columns={"Purchase ID": "Purchase Count",
                                          "Price_x": "Average Purchase Price", "Price_y": "Total Purchase Value"})
df_combined["Average Purchase Price"] = df_combined["Average Purchase Price"].astype(float).map("${:,.2f}".format)
df_combined["Total Purchase Value"] = df_combined["Total Purchase Value"].astype(float).map("${:,.2f}".format)
df_combined["Avg Total Purchase per Person"] = df_combined["Avg Total Purchase per Person"].astype(float).map("${:,.2f}".format)
df_combined


,Gender,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
0,Female,113,$3.20,$361.94,$4.47
1,Male,652,$3.02,"$1,967.64",$4.07
2,Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

### Generate a Age Demographics table by categorizing the existing players using the age bins.
1. Total Number of Players in Each Age Group
2. The Percentage of the Age Group 

In [113]:
bins = [0,9,14,19,24,29,34,39,100]
group_names = ["<10","10-14","15-19","20-24","25-29","30-34","35-39","40+"]
purchase_data["Age Group"]=pd.cut(purchase_data["Age"],bins,labels=group_names)
df_age_demo = (purchase_data.groupby(["Age Group"])["SN"].nunique().reset_index()).set_index("Age Group")
df_percent = df_age_demo / len(purchase_data["SN"].unique()) * 100
df_combined = (pd.merge(df_age_demo, df_percent, on="Age Group", how="outer")).rename(columns={"SN_x":"Total Count", "SN_y": "Percentage of Players"})
df_combined["Percentage of Players"]=df_combined["Percentage of Players"].astype(float).map("{:.2f}".format)
df_combined

,Total Count,Percentage of Players
Age Group,,
<10,17,2.95
10-14,22,3.82
15-19,107,18.58
20-24,258,44.79
25-29,77,13.37
30-34,52,9.03
35-39,31,5.38
40+,12,2.08


## Purchasing Analysis (Age)

In [116]:
bins = [0,9,14,19,24,29,34,39,100]
group_names = ["<10","10-14","15-19","20-24","25-29","30-34","35-39","40+"]
df_pa_id = (purchase_data.groupby(["Age Group"])["Purchase ID"].count().reset_index()).set_index("Age Group")
df_pa_avg_price = (purchase_data.groupby(["Age Group"])["Price"].mean().reset_index()).set_index("Age Group")
df_pa_total = (purchase_data.groupby(["Age Group"])["Price"].sum().reset_index()).set_index("Age Group")
df_pa_avg_person = (purchase_data.groupby(["Age Group"])["Price"].sum())/(purchase_data.groupby(["Age Group"])["SN"].nunique())
df_pa_avg_person = df_pa_avg_person.reset_index().set_index("Age Group")
df_pa_id["Average Purchase Price"] = df_pa_avg_price
df_pa_id["Total Purchase Value"] = df_pa_total
df_pa_id["Avg Total Purchase per Person"] = df_pa_avg_person
df_pa_id["Average Purchase Price"] = df_pa_id["Average Purchase Price"].astype(float).map("${:,.2f}".format)
df_pa_id["Total Purchase Value"] = df_pa_id["Total Purchase Value"].astype(float).map("${:,.2f}".format)
df_pa_id["Avg Total Purchase per Person"] = df_pa_id["Avg Total Purchase per Person"].astype(float).map("${:,.2f}".format)
df_combined = df_pa_id.reindex(["10-14","15-19","20-24","25-29","30-34","35-39","40+","<10"])
df_combined

,Purchase ID,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Group,,,,
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19
<10,23,$3.35,$77.13,$4.54


## Top Spenders

### Generate a summary table displaying purchasing summary for each player: 
1. Number of Purchase
2. Average Purchase Price
3. Total Purchase Value

In [161]:
df_sn = (purchase_data.groupby(["SN"])["Purchase ID"].count().reset_index()).set_index("SN")
df_sn_price = (purchase_data.groupby(["SN"])["Price"].mean().reset_index()).set_index("SN")
df_sn_total = (purchase_data.groupby(["SN"])["Price"].sum().reset_index()).set_index("SN")
df_sn ["Average Purchase Price"] = df_sn_price.round(2)
df_sn ["Total Purchase Value"] = df_sn_total.round(2)
df_sn ["Average Purchase Price"] = df_sn["Average Purchase Price"].map('${:,.2f}'.format)
df_sn ["Total Purchase Value"] = pd.to_numeric(df_sn ["Total Purchase Value"])
df_sn = df_sn.rename(columns={"Purchase ID": "Purchase Count"})
df_sn.sort_values(by="Total Purchase Value", ascending=False).head(5)

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,18.96
Idastidru52,4,$3.86,15.45
Chamjask73,3,$4.61,13.83
Iral74,4,$3.40,13.62
Iskadarya95,3,$4.37,13.10


## Most Popular Items & Most Profitable Items

### Generate a summary table to seek out the most POPULAR and PROFITABLE item by Groupying by: 
1. Item ID
2. Item Name
3. Number of Purchase
4. Item Price
5. Total Purchase Value

In [192]:
df_items = (purchase_data.groupby(["Item ID","Item Name"])["Purchase ID"].count().reset_index()).set_index("Item ID","Item Name")
df_items_price = (purchase_data.groupby(["Item ID","Item Name"])["Price"].mean().reset_index()).set_index("Item ID","Item Name")
df_combined = pd.merge(df_items, df_items_price, on=["Item ID","Item Name"], how="outer")
df_total = (purchase_data.groupby(["Item ID","Item Name"])["Price"].sum().reset_index()).set_index("Item ID","Item Name")
df_combined = pd.merge(df_combined,df_total,on=["Item ID","Item Name"],how="outer")
df_combined = df_combined.rename(columns={"Purchase ID":"Purchase Count", "Price_x":"Item Price", "Price_y":"Total Purchase Value"})
df_combined["Item Price"] = df_combined["Item Price"].map("${:.2f}".format)
df_combined["Total Purchase Value"] = df_combined["Total Purchase Value"].map("${:.2f}".format)
df_combined.sort_values(by="Purchase Count", ascending=False).head(5)

,Item Name,Purchase Count,Item Price,Total Purchase Value
Item ID,,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16
